# 환경 세팅 및 데이터셋 업로드

In [ ]:
#코랩 환경 준비 및 ultralytics 설치
!pip install ultralytics roboflow

In [ ]:
# 로보플로에서 api 활용한 데이터셋 다운로드(YOLOv8 포맷)
from roboflow import Roboflow

rf = Roboflow(api_key="bGEamJdSzr5rBrg9Sjne")
project = rf.workspace("kiosk-nmotv").project("osp-bao99")
version = project.version(4)
dataset = version.download("yolov8")

# YOLOv8 모델 학습시키기

In [ ]:
# -------yolov8 학습 코드--------
from ultralytics import YOLO
# 하이퍼파라미터를 변수로 저장 (이걸로 다른 셀에서도 접근 가능!)
epochs = 150
batch = 32                 # 배치 크기 (GPU 메모리에 따라 조절)

# 데이터셋 yaml 경로 확인 (Roboflow에서 다운로드된 경로)
data_yaml = dataset.location + "/data.yaml"

# 원하는 모델 선택 (yolov8n.pt, yolov8s.pt, yolov8m.pt, yolov8l.pt, yolov8x.pt)
model = YOLO('yolov8s.pt') # 해보고 과적합되면 yolov8n.pt로 시도해보기

# 학습 시작
results = model.train(
    data=data_yaml,     # 데이터셋 yaml 파일 경로
    epochs=epochs,
    batch=batch,
    imgsz=640,          # 입력 이미지 크기
    name='yolov8s_custom', # 결과 저장 폴더명
    exist_ok=True,

    # 아이스크림 UI 다양성을 위한 증강
    # hsv_h=0.1,      # 색상 변화 (UI 색상 다양화)
    # hsv_s=0.8,      # 채도 변화
    # hsv_v=0.5,      # 밝기 변화 (화면 밝기)
    # scale=0.8,      # 크기 변화
    # translate=0.3,  # 위치 변화
    # fliplr=0.5,     # 좌우 반전
    # mosaic=1.0,     # 모자이크 (매우 효과적)
    # mixup=0.3,      # 믹스업
    # erasing=0.2,    # 일부 영역 지우기
)

val_results = model.val(
    data = data_yaml,
    split = 'test',
    plots = True,
    save_json = True,
    exist_ok=True
)

# 추론 테스트 코드

In [ ]:
# 학습된 모델 로드
model = YOLO('/content/runs/detect/yolov8s_custom/weights/best.pt')

# 이미지 추론
import glob

# test 이미지 폴더 경로
test_img_dir = '/content/OSP-4/test/images/'

# 모든 이미지 파일 리스트 얻기 (jpg, png 등)
img_list = glob.glob(test_img_dir + '*.*')

for img_path in img_list:
    results = model(img_path)
    results[0].show()  # 바운딩박스, 클래스명 표시

# **학습 성능 시각화 코드**

가장 최근에 학습된 모델의 results.csv(학습결과) 자동으로 불러옴

In [ ]:
import os
import glob
import pandas as pd

# 1. yolov8s_custom* 폴더 중 최신 폴더 찾기
base_dir = 'runs/detect'
pattern = os.path.join(base_dir, 'yolov8s_custom*')
folders = glob.glob(pattern)

if not folders:
    raise FileNotFoundError("No yolov8s_custom* folders found in runs/detect/")
else:
    # 수정: 폴더 생성(수정) 시간 기준으로 최신 폴더 선택
    folders.sort(key=lambda x: os.path.getmtime(x))
    latest_folder = folders[-1]

# 2. 최신 폴더의 results.csv 경로
results_csv_path = os.path.join(latest_folder, 'results.csv')
if not os.path.exists(results_csv_path):
    raise FileNotFoundError(f"results.csv not found in {latest_folder}")

Box Loss 그래프: 학습/검증 박스 손실 추이

Precision-Recall 그래프: 정밀도와 재현율 변화
  (재현율 : 실제 정답 객체 중 모델이 정답으로 검출한 객체 수 -> 그냥 정확도로 생각해도 될듯)

mAP 그래프: 0.5 IoU, 0.5-0.95 IoU 기준 mAP 변화
  (정확도,재현율의 관계 평가 -> mAP값이 높으면 객체 탐지율이 높은 것)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv(results_csv_path) # 위에서 구한 결과 csv 파일 경로 불러옴

plt.figure(figsize=(10, 10))

# 1. Train/Val Box Loss
plt.subplot(3, 1, 1)
plt.plot(df['epoch'], df['train/box_loss'], label='Train Box Loss', marker='o')
plt.plot(df['epoch'], df['val/box_loss'], label='Val Box Loss', marker='o')
plt.title('Train and Validation Box Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# 2. Precision and Recall (0~100%)
plt.subplot(3, 1, 2)
plt.plot(df['epoch'], df['metrics/precision(B)']*100, label='Precision (%)', marker='o')
plt.plot(df['epoch'], df['metrics/recall(B)']*100, label='Recall (%)', marker='o')
plt.title('Precision and Recall')
plt.xlabel('Epoch')
plt.ylabel('Percentage (%)')
plt.ylim(0, 100)  # y축을 0~100으로 고정
plt.legend()

# 3. mAP@0.5 and mAP@0.5:0.95 (0~100%)
plt.subplot(3, 1, 3)
plt.plot(df['epoch'], df['metrics/mAP50(B)']*100, label='mAP@0.5 (%)', marker='o')
plt.plot(df['epoch'], df['metrics/mAP50-95(B)']*100, label='mAP@0.5:0.95 (%)', marker='o')
plt.title('Mean Average Precision (mAP)')
plt.xlabel('Epoch')
plt.ylabel('Percentage (%)')
plt.ylim(0, 100)  # y축을 0~100으로 고정
plt.legend()

plt.tight_layout()
plt.show()


모델 성능 수치화해서 출력
-> 이거를 깃헙에 올릴 때 커밋 메시지에 포함해 주세요!
예) 모델 성능 : Epoch 150, batch 16 => mAP@0.5=82.1%, mAP@0.5:0.95=63.4%, Precision=87.2%, Recall=71.5%

In [ ]:
from ultralytics import YOLO

# 학습된 모델 로드
model = YOLO('runs/detect/yolov8s_custom/weights/best.pt')

# test셋에서 평가
metrics = val_results

# 주요 성능 지표 명시적으로 출력
print(f"mAP@0.5: {metrics.box.map50*100:.2f}%")
print(f"mAP@0.5:0.95: {metrics.box.map*100:.2f}%")
print(f"Precision: {metrics.box.mp*100:.2f}%")
print(f"Recall: {metrics.box.mr*100:.2f}%")


# 학습 결과 내 드라이브에 자동 저장
best.pt 파일과, 에폭, batch값, 성능 지표4가지값 모두 자동 저장되도록 함.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from ultralytics import YOLO
import os

# 드라이브 내 저장 경로
save_base = '/content/drive/MyDrive/OSS_yolo_weights'
save_dir = os.path.join(save_base, f'ep{epochs}_batch{batch}')
os.makedirs(save_dir, exist_ok=True)

# best.pt, data.yaml 경로
best_pt_path = 'runs/detect/yolov8s_custom/weights/best.pt'
data_yaml_path = 'runs/detect/yolov8s_custom/data.yaml'

# 모델 로드 및 평가
model = YOLO(best_pt_path)

# 성능지표 텍스트 생성 (자동 추출)
performance_text = f"""mAP@0.5: {metrics.box.map50*100:.2f}%
mAP@0.5:0.95: {metrics.box.map*100:.2f}%
Precision: {metrics.box.mp*100:.2f}%
Recall: {metrics.box.mr*100:.2f}%
Epoch: {epochs}
Batch size: {batch}
"""

# best.pt 복사
import shutil
shutil.copy(best_pt_path, os.path.join(save_dir, 'best.pt'))

# 성능지표 txt 저장
metrics_save_path = os.path.join(save_dir, 'performance.txt')
with open(metrics_save_path, 'w') as f:
    f.write(performance_text)

print(f"best.pt와 성능지표가 {save_dir}에 저장되었습니다.")
